In [17]:
# python packages
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import numpy as np
import pandas as pd
from datetime import datetime as dt
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import json
import xarray as xr

# my imports
import prop_data_prep as pdp
import prop_nn_models as pnn
import prop_nn_functions as pnf
import prop_MCDO_functions as MCDO
import prop_visualizations as viz
from prop_mms_data_prep import write_toDf, sweep

## MODEL LOADING

In [20]:
# model params
# inputs t0, te, imef_data
# assume nans removed

# load model
model_floc = "prop_models/"  # directory to save model
model_fname = "ANN_complete_EXYZ"  # sample model
model_path = model_floc + model_fname  # full model path

# load train/test dataset (want to use test data in futr maybe?)
imef_data = pd.read_pickle("prop_models/complete_training_df")
time_indx = imef_data.index.values[100:160]  # "test times"

# load hyperparameter configuration
f = open(model_path + "_config.json")
CONFIG = json.load(f)

# initialize the model
model = pnn.FeedForwardNN_MCDO(
    CONFIG["num_features"],
    CONFIG["seq_len"],
    CONFIG["hidden_size"],
    CONFIG["output_size"],
    CONFIG["dropout_prob"],
)
model.load_state_dict(torch.load(model_floc + model_fname))

predictors = [
    "OMNI_IMF",
    "OMNI_Vx",
    "OMNI_Vy",
    "OMNI_Vz",
    "OMNI_SYM_H",
    "L",
    "MLAT",
    "MLT",
]  # predictors, features
target = ["EX", "EY", "EZ"]

# separate swolar wind (swi) and location (loc) predictors
#   - this will be used to index and assemble matrix correctly
swi_predictors = predictors[:5]  # solar wind predictors
loc_predictors = predictors[5:]  # location (L,MLT, MLAT) predictors

## CREATE DEISGN MATRIX


In [24]:
"""
Assemble Design Matrix
for Izzak's code, see
    - imef/efield/model_creation/NN_functions.py 
        - (get_NN_inputs function!)

"""

design_matrix = []  # list to hold desigh matrix
time_matrix = []  # list to hold times assoc. with matrix values

for ix in range(0, len(time_indx)):
    ix_matrix_row = []  # matrix row at loop index (ix)

    # get time intervals
    # `- (!) should be end??
    time_intervals = pd.date_range(end=time_indx[ix], freq="5T", periods=60)
    # print('index',time_indx)
    # print('intrvl',time_intervals)
    # try:
    # write predictors at ix to row

    # loop through each swind predictor and append it to matrix
    for swi_var in swi_predictors:
        # pull predictor from dataset as xarray
        swi_xarr = imef_data[swi_var].to_xarray()

        # split data in interval format and append to design matrix row
        ix_arr = swi_xarr.sel(time=time_intervals, method="nearest").values.tolist()
        ix_matrix_row += ix_arr  

    # get location data (L, MLT, MLAT) and assemble into matrix format
    loc_data = np.array(
        [
            imef_data["L"].values[ix],
            np.cos(np.pi / 12 * imef_data["MLT"].values[ix]),
            np.sin(np.pi / 12 * imef_data["MLT"].values[ix]),
            np.cos(imef_data["MLAT"].values[ix]),
            np.sin(imef_data["MLAT"].values[ix]),
        ]
    )

    # write location data (np array) at ix to row (list)
    ix_matrix_row += loc_data.tolist()

    # append row to design matrix
    # print("row",ix_matrix_row)
    design_matrix.append(ix_matrix_row)

    # append times to time matrix
    time_matrix.append(time_indx[ix])

# torch tensor design matrix
design_matrix_tensor = torch.tensor(design_matrix)

# except Exception as ex:
#     pass

In [30]:
"""
for Izzak's code, see
    - imef/data_manipulation.py
    - visualizations/plot_nc_data.py
    - imef/efield/model_creation/NN_functions.py
    - imef/data/database.py
        - **(predict_efield_and_potential function!)

"""

number_of_inputs = len(swi_predictors)  # number of inputs/predictors

# unsure how to adjust this for all predicors? is this grabbing row or column?
# izzak used:
#   >> base_kp_values = design_matrix_tensor[-1].clone()
# my variation (I don't know what to do for this):
base_values = design_matrix_tensor[-1].clone()

# izzak used:
#   >> size_of_input_vector = 60 * number_of_inputs + 3
# my variation:
size_of_input_vector = 60 * number_of_inputs + 5

# I THINK this is izzak's way of setting up the matrix to be accepted
# by the model, but...its not clear.
for L in range(4, 11):
    for MLT in range(0, 24):
        new_row = base_values.clone()
        new_row[-3] = L
        new_row[-2] = np.cos(np.pi / 12 * MLT)
        new_row[-1] = np.sin(np.pi / 12 * MLT)
        even_newer_row = torch.empty((1, size_of_input_vector))
        even_newer_row[0] = new_row
        if L == 4 and MLT == 0:
            all_locations = even_newer_row
        else:
            all_locations = torch.cat((all_locations, even_newer_row))

model.eval()
with torch.no_grad():
    pred = model(all_locations)

RuntimeError: shape '[168, 480]' is invalid for input of size 51240

# Plotting (TBD)

In [26]:
base_values = design_matrix_tensor[design_matrix_tensor.columns[-3:]]

AttributeError: 'Tensor' object has no attribute 'columns'